In [7]:
import pubchempy as pcp

# c_list = pcp.get_compounds("C6H12", "formula", record_type="3d")
search_name = "Cyclopropene"
c_list = pcp.get_compounds(search_name, "name", record_type="3d")

c_list_clean = []
for c in c_list:
    if c.cid < 100000:
        c_list_clean.append(c)

for c in c_list_clean:
    print(c.cid, c.iupac_name, c.synonyms[0])
# c_list_clean.pop()
# for c in c_list_clean:
#     print(c.cid, c.iupac_name, c.synonyms[0])

In [8]:
from pyscf.geomopt.geometric_solver import optimize
from pyscf import gto, scf
import json
import copy
import json

with open("../cadft/utils/mol.json") as f:
    data = json.load(f)

dict_ = {}

for c in c_list_clean:
    name = search_name
    # if name is None:
    #     name = c.synonyms[0]
    # if name is None:
    #     print("No name for", c.cid)
    #     continue
    name = name.lower().replace(" ", "_")

    molecular = []
    for c_atom in c.atoms:
        atom = []
        atom.append(c_atom.element)
        atom.append(round(c_atom.x, 4))
        atom.append(round(c_atom.y, 4))
        atom.append(round(c_atom.z, 4))
        molecular.append(atom)

    mol = gto.M(atom=molecular, basis="ccpvdz")
    mdft = mol.KS()
    mdft.xc = "b3lyp"
    mdft.kernel()
    mol_eq = optimize(mdft, maxsteps=100, verbose=0)

    molecular_new = copy.deepcopy(molecular)
    for i, atom in enumerate(molecular_new):
        atom[1] = mol_eq.atom_coords(unit="ANG")[i][0]
        atom[2] = mol_eq.atom_coords(unit="ANG")[i][1]
        atom[3] = mol_eq.atom_coords(unit="ANG")[i][2]

    dict_[name] = molecular

data.update(dict_)

with open("../cadft/utils/mol.json", "w") as f:
    json.dump(data, f)